In [1]:
import itertools
from collections import defaultdict
import random
import pandas as mypanda
from collections import Counter
import numpy as mynumpy

    


In [2]:
fh=open(r'dataset_NB.txt','r')
stopping_wordings = ['your','you','yet','have','has','how','had','else','each','does','did','can','but','are','any','and','may','were','there','the','who']


In [3]:
def funcclean(fh):
    axis_y=[]
    mainfile=fh.readlines()
    i=0
    while(i<len(mainfile)):
        fileword=mainfile[i]
        fileword=fileword.strip()
        mainfile[i]=fileword.lower()
        axis_y.append(int(fileword[-1]))
        i=i+1
    funcclean_file=[]
    terms=[]
    
    for fileword in mainfile:
        x=fileword.split()
        for term in x :
            if(term.isalpha() and term not in stopping_wordings and len(term)>2):
                terms.append(term)
        funcclean_file.append(terms)
        terms=[]
    
    return funcclean_file , axis_y



In [4]:
mainfile,axis_y=funcclean(fh)

In [5]:
def validation_check(mainfile,axis_y):
    mainfile=mynumpy.array(mainfile)
    axis_y=mynumpy.array(axis_y)
    index_check = mynumpy.arange(mainfile.shape[0])
    mynumpy.random.shuffle(index_check)
    mainfile=list(mainfile[index_check])
    axis_y=list(axis_y[index_check])
    interim=[]
    axis_y_interim = []
    practice=[]
    axis_y_practice=[]
    trial=[]
    axis_y_trial= []
    value_current=0
    i=0
    while(i<7):
        if(i!=6):
            trial.append(mainfile[value_current:value_current+int(len(mainfile)/7)])
            axis_y_trial.append(axis_y[value_current:value_current+int(len(mainfile)/7)])
        else:
            trial.append(mainfile[value_current:])
            axis_y_trial.append(axis_y[value_current:])
            
        if(i!=0):
            interim.append(mainfile[0:value_current])
            axis_y_interim.append(axis_y[0:value_current])
        if(i!=6):
            interim.append(mainfile[value_current+int(len(mainfile)/7):])
            axis_y_interim.append(axis_y[value_current+int(len(mainfile)/7):])
        
        interim=list(itertools.chain.from_iterable(interim))
        axis_y_interim=list(itertools.chain.from_iterable(axis_y_interim))
        practice.append(interim)
        axis_y_practice.append(axis_y_interim)
        value_current=value_current+int(len(mainfile)/7)
        interim=[]
        axis_y_interim=[]
        i=i+1
    return practice,axis_y_practice,trial,axis_y_trial



In [6]:
def Theorem_Bayes_prob( s ,apm ,apm_frequency):
    prime=apm[s]/apm_frequency   #p(x1/c1)
    return prime

def Theorem_Bayes_fig( s ,fig,frequency_fig):
    quant=fig[s]/frequency_fig
    return quant


In [7]:
def dict_make(practice,axis_y):
    axis_y=mynumpy.array(axis_y)
    setting_apm=()
    fig_setting=()
    
    apm_position=mynumpy.where(axis_y==0)[0]
    fig_position=mynumpy.where(axis_y==1)[0]
    apm=Counter(practice[apm_position[0]])
    fig=Counter(practice[fig_position[0]])
    count_apm=len(apm_position)
    fig_count=len(fig_position)
    i=1
    while(i<count_apm):
        apm.update(Counter(practice[apm_position[i]]))
        i=i+1
    i=1
    while(i<fig_count):
        fig.update(Counter(practice[fig_position[i]]))
        i=i+1
    return count_apm,fig_count,apm,fig

In [8]:
practice,axis_y_practice,trial,axis_y_trial=validation_check(mainfile,axis_y)
axis_y_practice=mynumpy.array(axis_y_practice)
validity=[]
i=0
while(i<7):
    invalid=0
    count_apm , fig_count , apm , fig = dict_make(practice[i],axis_y_practice[i])
    apm_frequency = sum(apm.values())
    frequency_fig = sum(fig.values())
    

    j=0
    while(j<len(trial[i])):
        fig_ans=fig_count / (count_apm+fig_count)         
        quant=0
        k=0
        
     
        while(k<len(trial[i][j])):
            quant = Theorem_Bayes_fig( trial[i][j][k] ,fig, frequency_fig)           
            fig_ans*=quant              #P(C1/x)=(P(x/C1)*P(c1))                 #P(x/C1)=P(x1/C1)*P(x2/C1).....     
            k=k+1

        answer_apm=count_apm / (count_apm+fig_count)         
        prime=0
        k=0
        
       
        while(k<len(trial[i][j])):
            prime= Theorem_Bayes_prob( trial[i][j][k] , apm, apm_frequency)
            answer_apm*=prime     
            k=k+1
            
      
        if(answer_apm>fig_ans and axis_y_trial[i][j]==1):
            invalid+=1
        elif(answer_apm<fig_ans and axis_y_trial[i][j]==0):
            invalid+=1
        j=j+1
    acc=(1-(invalid/len(axis_y_trial[i])))*100
    print(acc)
    validity.append(acc)
    i=i+1

C:\Users\AbhinavBhattacharjee\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


90.84507042253522
93.66197183098592
92.25352112676056
90.14084507042254
90.84507042253522
94.36619718309859
92.56756756756756


In [9]:
account_average=sum(validity)/len(validity)
print("Average of accuracy=",account_average)
print("maximum accuracy=",max(validity))

Average of accuracy= 92.09717766055795
maximum accuracy= 94.36619718309859
